In [1]:
# Import Libraries
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision.transforms as transforms
import torchvision.datasets as datasets
from torch.utils.data import DataLoader
from torchvision.models import resnet50
from tqdm import tqdm

In [2]:
# Mount Google Drive to access the dataset
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [3]:
# Set the device
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [4]:
# Define transforms for the data
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))  # normalize to [-1, 1]
])

In [5]:
# Define dataset and data loader for training
train_data_dir = '/content/gdrive/MyDrive/Lemon_quality_classification/lemon_dataset/train'
train_dataset = datasets.ImageFolder(train_data_dir, transform=transform)
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)

In [6]:
# Define dataset and data loader for validation
val_data_dir = '/content/gdrive/MyDrive/Lemon_quality_classification/lemon_dataset/val'
val_dataset = datasets.ImageFolder(val_data_dir, transform=transform)
val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False)

In [7]:
# Define ResNet model
model = resnet50(pretrained=True)
num_features = model.fc.in_features
num_classes = len(train_dataset.classes)
model.fc = nn.Linear(num_features, num_classes)  # Replace the fully connected layer
model = model.to(device)

/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/resnet50-0676ba61.pth" to /root/.cache/torch/hub/checkpoints/resnet50-0676ba61.pth
100%|██████████| 97.8M/97.8M [00:00<00:00, 183MB/s]


In [8]:
# Define loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

In [9]:
# Training loop
num_epochs = 5
checkpoint_interval = 2  # Save the model every 2 epochs
checkpoint_path = '/content/gdrive/MyDrive/Lemon_quality_classification/checkpoints/model.pth'

for epoch in range(num_epochs):
    # Training phase
    model.train()
    running_loss = 0.0
    correct_predictions = 0
    total_predictions = 0

    progress_bar = tqdm(train_loader, desc=f'Epoch {epoch+1}/{num_epochs} - Training', unit='batch')

    for images, labels in progress_bar:
        images = images.to(device)
        labels = labels.to(device)

        optimizer.zero_grad()

        # Forward pass
        outputs = model(images)
        loss = criterion(outputs, labels)

        # Backward pass and optimize
        loss.backward()
        optimizer.step()

        running_loss += loss.item()

        _, predicted = torch.max(outputs, 1)
        total_predictions += labels.size(0)
        correct_predictions += (predicted == labels).sum().item()

        progress_bar.set_postfix({'Loss': running_loss / len(train_loader), 'Train Accuracy': f'{100 * correct_predictions / total_predictions:.2f}'})

    # Validation phase
    model.eval()
    val_loss = 0.0
    val_correct_predictions = 0
    val_total_predictions = 0

    val_progress_bar = tqdm(val_loader, desc=f'Epoch {epoch+1}/{num_epochs} - Validation', unit='batch')

    with torch.no_grad():
        for val_images, val_labels in val_progress_bar:
            val_images = val_images.to(device)
            val_labels = val_labels.to(device)

            val_outputs = model(val_images)
            val_loss += criterion(val_outputs, val_labels).item()

            _, val_predicted = torch.max(val_outputs, 1)
            val_total_predictions += val_labels.size(0)
            val_correct_predictions += (val_predicted == val_labels).sum().item()

            val_progress_bar.set_postfix({'Val Loss': val_loss / len(val_loader), 'Val Accuracy': f'{100 * val_correct_predictions / val_total_predictions:.2f}'})

    if (epoch + 1) % checkpoint_interval == 0:
        # Save the model checkpoint
        torch.save(model.state_dict(), checkpoint_path)
        print(f"Checkpoint saved at epoch {epoch+1}")

print("Training completed!")

Epoch 2/5 - Validation: 100%|██████████| 12/12 [00:03<00:00,  3.88batch/s, Val Loss=0.0183, Val Accuracy=99.47]


Checkpoint saved at epoch 2


Epoch 4/5 - Validation: 100%|██████████| 12/12 [00:03<00:00,  3.45batch/s, Val Loss=0.016, Val Accuracy=99.47]


Checkpoint saved at epoch 4


Epoch 5/5 - Validation: 100%|██████████| 12/12 [00:03<00:00,  3.93batch/s, Val Loss=0.0126, Val Accuracy=99.47]

Training completed!


In [10]:
# Save the final trained model
final_model_path = '/content/gdrive/MyDrive/Lemon_quality_classification/model.pth'
torch.save(model.state_dict(), final_model_path)